<a href="https://colab.research.google.com/github/bahram3000/test_balancer_app/blob/main/manage_positions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qqq web3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.5/587.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.3/340.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.8/175.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 37.6 MB/s eta 0:00:00


In [2]:
from web3 import Web3
from eth_account import Account
import requests
import json
import time
import decimal
import numpy as np
import math

In [3]:
from web3 import exceptions
from eth_abi import encode

In [4]:
RPC = "https://eth-sepolia.g.alchemy.com/v2/1d3m81eR2Kd2mrpuH6-bR"
w3 = Web3(Web3.HTTPProvider(RPC))

# کلید خصوصی والت تست‌نت
PRIVATE_KEY = "75074ee5760d9640d993d09bc6e969b37cadcc2c83195886d43b3e86b2b8b6da"
account = Account.from_key(PRIVATE_KEY)
my_wallet_address = account.address

print("Connected:", w3.is_connected())
print("Your address:", my_wallet_address)

Connected: True
Your address: 0x7868EDEfC36A74eba220C75aCFB05c59a4e55822


In [5]:
def fetch_implementation_address(contract_address: str, api_key: str, inp_chain_id: int):
    try:
        contract_address = Web3.to_checksum_address(contract_address)
    except ValueError:
        return None

    url = "https://api.etherscan.io/v2/api"
    params = {
        "module": "contract",
        "action": "getsourcecode", # اکشن تغییر کرد
        "address": contract_address,
        "apikey": api_key,
        "chainid": inp_chain_id
    }

    try:
        resp = requests.get(url, params=params, timeout=10)
        resp.raise_for_status()
        data = resp.json()

        if data.get("status") == "1" and data["result"]:
            result = data["result"][0]
            implementation_address = result.get('Implementation')

            if implementation_address and implementation_address != '0x0000000000000000000000000000000000000000':
                return Web3.to_checksum_address(implementation_address)

    except Exception as e:
        print(f"Error checking implementation for {contract_address}: {e}")

    return None

In [6]:
def fetch_abi_retry(contract_address: str, api_key: str, inp_chain_id: int, retries=3):
    try:
        initial_address = Web3.to_checksum_address(contract_address)
    except ValueError:
        return None

    implementation_address = fetch_implementation_address(initial_address, api_key, inp_chain_id)

    addresses_to_check = []
    if implementation_address:
        addresses_to_check.append(implementation_address)
    addresses_to_check.append(initial_address)

    for current_addr in addresses_to_check:
        url = "https://api.etherscan.io/v2/api"
        params = {
            "module": "contract",
            "action": "getabi",
            "address": current_addr,
            "apikey": api_key,
            "chainid": inp_chain_id
        }

        for attempt in range(retries):
            try:
                resp = requests.get(url, params=params, timeout=10)
                resp.raise_for_status()
                data = resp.json()

                if data.get("status") == "1":
                    print(f"Successfully fetched ABI from: {current_addr}")
                    return json.loads(data["result"])


            except requests.exceptions.RequestException as e:
                time.sleep(1)

        time.sleep(0.5)

    return None

In [7]:
etherscan_apikey = "N68HP1FGB4UQQFBHZQUJIM12HVYWJME96M"
CHAIN_ID = 11155111  # Sepolia

In [8]:
position_manager_address = "0x1238536071E1c677A632429e3655c799b22cDA52"
position_manager_address = w3.to_checksum_address(position_manager_address)
position_manager_abi = fetch_abi_retry(position_manager_address,etherscan_apikey,CHAIN_ID)
position_manager_contract = w3.eth.contract(address=position_manager_address, abi=position_manager_abi)

Successfully fetched ABI from: 0x1238536071E1c677A632429e3655c799b22cDA52


In [9]:
def check_liquidity_v3_nft(inp_wallet_address,inp_postion_manger_contract):
  pm_contract=inp_postion_manger_contract
  address=inp_wallet_address
  nfts=[]
  try:
    balance = pm_contract.functions.balanceOf(address).call()
    print(f"تعداد NFT پوزیشن‌ها: {balance}")


    if balance > 0:
        for i in range(balance):
            time.sleep(1)
            token_id = pm_contract.functions.tokenOfOwnerByIndex(address, i).call()

            pos = pm_contract.functions.positions(token_id).call()
            dct={'token_id':token_id,'token0':pos[2],'token1':pos[3],'fee_tier':pos[4],'liquidity':pos[7],'tick_range':[pos[5],pos[6]],'fee0':pos[10],'fee1':pos[11]}
            nfts.append(dct)
        return nfts
    else:
        print("lp nft not found")

  except Exception as e:
    print(f"error in read data --> {e}")

In [10]:
V3_LPS=check_liquidity_v3_nft(my_wallet_address,position_manager_contract)
V3_LPS

تعداد NFT پوزیشن‌ها: 2


[{'token_id': 221855,
  'token0': '0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238',
  'token1': '0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14',
  'fee_tier': 500,
  'liquidity': 535164611370740,
  'tick_range': [187930, 187940],
  'fee0': 0,
  'fee1': 0},
 {'token_id': 221856,
  'token0': '0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238',
  'token1': '0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14',
  'fee_tier': 100,
  'liquidity': 19436422026195112,
  'tick_range': [187953, 187955],
  'fee0': 0,
  'fee1': 0}]

In [11]:
v3factoryaddress="0x0227628f3F023bb0B980b67D528571c95c6DaC1c"
factory_address = w3.to_checksum_address(v3factoryaddress)
factory_abi =fetch_abi_retry(factory_address,etherscan_apikey,CHAIN_ID)
factory=w3.eth.contract(address=factory_address, abi=factory_abi)

Successfully fetched ABI from: 0x0227628f3F023bb0B980b67D528571c95c6DaC1c


In [12]:
def calculate_price_from_sqrt_price(sqrt_price_raw: float, decimal_0: int, decimal_1: int) -> float:
    normalized_sqrt_price = sqrt_price_raw / (2 ** 96)
    raw_price_math = normalized_sqrt_price ** 2
    decimal_adjustment = decimal_0 - decimal_1
    final_price = raw_price_math * (10 ** decimal_adjustment)
    return final_price

In [13]:
def pool_current_info(inp_pool_contract):
  token0_address = inp_pool_contract.functions.token0().call()
  token1_address = inp_pool_contract.functions.token1().call()
  slot0 = inp_pool_contract.functions.slot0().call()
  liquidity = inp_pool_contract.functions.liquidity().call()
  fee = inp_pool_contract.functions.fee().call()
  tickspacing = inp_pool_contract.functions.tickSpacing().call()
  return {'token0_address':token0_address,'token1_address':token1_address,'sqrtprice':slot0[0],'current_tick':slot0[1],'liquidity':liquidity,'fee_tier':fee,'tick_spacing':tickspacing}

In [38]:
# Helper function to ensure we have a valid timestamp for deadline
def get_deadline(ttl=1200):
    """Returns current timestamp + ttl (time to live in seconds)"""
    return int(time.time()) + ttl

def send_signed_transaction(w3_instance: Web3, signed_tx, wait_for_receipt=True) -> str:
    """
    Sends a signed transaction and optionally waits for the receipt.
    Returns the transaction hash as a Hex string.
    """
    tx_hash = w3_instance.eth.send_raw_transaction(signed_tx.raw_transaction)

    if wait_for_receipt:
        print(f"Waiting for transaction receipt: {w3_instance.to_hex(tx_hash)}")
        w3_instance.eth.wait_for_transaction_receipt(tx_hash)

    return w3_instance.to_hex(tx_hash)

def handle_approve(w3_instance: Web3, input_token_contract, swap_router_address, amount_in_wei, inp_wallet, inp_private_key, nonce) -> int:
    """
    Checks token allowance and approves the router if necessary.
    Returns the next nonce to be used.
    """
    current_allowance = input_token_contract.functions.allowance(inp_wallet, swap_router_address).call()

    if current_allowance < amount_in_wei:
        print(f"Insufficient allowance. Approving...")

        # Get EIP-1559 gas parameters
        latest_block = w3_instance.eth.get_block('latest')
        base_fee_per_gas = latest_block['baseFeePerGas']
        # Fallback for priority fee if chain doesn't support fee history well
        max_priority_fee_per_gas = w3_instance.eth.max_priority_fee if w3_instance.eth.max_priority_fee else w3_instance.to_wei(1, 'gwei')
        max_fee_per_gas = base_fee_per_gas + 2 * max_priority_fee_per_gas

        # Build approve transaction
        approve_tx_build = input_token_contract.functions.approve(
            swap_router_address,
            amount_in_wei
        ).build_transaction({
            "from": inp_wallet,
            "nonce": nonce,
            "maxFeePerGas": max_fee_per_gas,
            "maxPriorityFeePerGas": max_priority_fee_per_gas
        })

        # Estimate gas for approval instead of hardcoding
        try:
            est_gas = w3_instance.eth.estimate_gas(approve_tx_build)
            approve_tx_build['gas'] = int(est_gas * 1.1) # Add 10% buffer
        except:
             approve_tx_build['gas'] = 80000 # Fallback if estimation fails

        signed_approve = w3_instance.eth.account.sign_transaction(approve_tx_build, inp_private_key)

        # Send and wait for confirmation
        send_signed_transaction(w3_instance, signed_approve, wait_for_receipt=True)
        print("Approve confirmed.")

        # Return incremented nonce for the next transaction
        return nonce + 1

    print("Allowance sufficient.")
    return nonce

def make_swap(w3_instance: Web3, input_token_address, output_token_address,
              amount_input: float, inp_wallet, inp_pool_address,
              swap_router_address, mode: str, inp_private_key, slippage: float = 0.01):

    # Checksum addresses
    input_token_address = w3_instance.to_checksum_address(input_token_address)
    output_token_address = w3_instance.to_checksum_address(output_token_address)
    pool_address = w3_instance.to_checksum_address(inp_pool_address)
    swap_router_address = w3_instance.to_checksum_address(swap_router_address)

    # 1. Fetch ABIs and Contracts
    # Note: Using a helper fetch_abi function assumed to be defined externally
    # It is better to use the specific token address for fetching ABI, not a hardcoded one
    erc20_abi = fetch_abi_retry(input_token_address, etherscan_apikey, CHAIN_ID)
    input_token_contract = w3_instance.eth.contract(address=input_token_address, abi=erc20_abi)

    swap_router_abi = fetch_abi_retry(swap_router_address, etherscan_apikey, CHAIN_ID)
    swap_router_contract = w3_instance.eth.contract(address=swap_router_address, abi=swap_router_abi)

    pool_abi = fetch_abi_retry(pool_address, etherscan_apikey, CHAIN_ID)
    pool_contract = w3_instance.eth.contract(address=pool_address, abi=pool_abi)

    # 2. Get Pool Info
    # Assumes pool_current_info returns: {'token0_address': ..., 'token1_address': ..., 'current_tick': ..., 'fee_tier': ...}
    pool_info = pool_current_info(pool_contract)

    token0_address = w3_instance.to_checksum_address(pool_info['token0_address'])
    token1_address = w3_instance.to_checksum_address(pool_info['token1_address'])

    # Validate tokens match the pool
    if not ((input_token_address == token0_address and output_token_address == token1_address) or
            (input_token_address == token1_address and output_token_address == token0_address)):
        print("Error: Input or output token address does not match the pool's tokens.")
        return False

    # 3. Get Decimals
    # Creating contracts just to read decimals.
    # We can reuse input_token_contract for one of them.
    token0_contract = w3_instance.eth.contract(address=token0_address, abi=erc20_abi)
    token1_contract = w3_instance.eth.contract(address=token1_address, abi=erc20_abi)

    decimal0 = token0_contract.functions.decimals().call()
    decimal1 = token1_contract.functions.decimals().call()

    # Calculate Amount In Wei
    input_decimals = decimal0 if input_token_address == token0_address else decimal1
    output_decimals = decimal1 if input_token_address == token0_address else decimal0

    amount_in_wei = int(amount_input * (10**input_decimals))

    # 4. Calculate Minimum Output (Slippage Protection)
    # Price is typically expressed as Token1 per Token0 (if using standard V3 math)
    # Assumes calculate_price_by_tick returns Price of Token0 in terms of Token1
    current_price = calculate_price_by_tick(pool_info['current_tick'], decimal0, decimal1)

    amountout_human = 0.0

    if input_token_address == token0_address:
        # Input is Token0 -> Output is Token1
        amountout_human = amount_input * current_price
    else:
        # Input is Token1 -> Output is Token0
        amountout_human = amount_input * (1 / current_price)

    # [FIX] Use output_decimals for calculating output wei
    amountout_min_wei = int(amountout_human * (10**output_decimals) * (1 - slippage))

    # 5. Handle Approve (Nonce management)
    # Get initial nonce
    nonce = w3_instance.eth.get_transaction_count(inp_wallet)

    # This might increment nonce if an approve tx is sent
    nonce = handle_approve(w3_instance, input_token_contract, swap_router_address, amount_in_wei, inp_wallet, inp_private_key, nonce)

    # 6. Prepare Swap Params
    # [FIX] Added 'deadline' which is required by ISwapRouter
    params = {
        "tokenIn": input_token_address,
        "tokenOut": output_token_address,
        "fee": pool_info['fee_tier'],
        "recipient": inp_wallet,
        "deadline": get_deadline(), # Expiry time (now + 1200s)
        "amountIn": amount_in_wei,
        "amountOutMinimum": amountout_min_wei,
        "sqrtPriceLimitX96": 0
    }

    # 7. Gas Parameters
    latest_block = w3_instance.eth.get_block('latest')
    base_fee_per_gas = latest_block['baseFeePerGas']
    max_priority_fee_per_gas = w3_instance.eth.max_priority_fee if w3_instance.eth.max_priority_fee else w3_instance.to_wei(1, 'gwei')
    max_fee_per_gas = base_fee_per_gas + 2 * max_priority_fee_per_gas

    # Build Transaction Dictionary
    tx_build_params = {
        "from": inp_wallet,
        "nonce": nonce, # Use the managed nonce
        "maxFeePerGas": max_fee_per_gas,
        "maxPriorityFeePerGas": max_priority_fee_per_gas
    }

    # 8. Simulate or Execute
    try:
        # Estimate gas to see if it reverts
        swap_tx_estimate = swap_router_contract.functions.exactInputSingle(params).build_transaction(tx_build_params)
        est_gas_need = w3_instance.eth.estimate_gas(swap_tx_estimate)

        # Add buffer to estimated gas
        tx_build_params['gas'] = int(est_gas_need * 1.1)

    except exceptions.ContractLogicError as e:
        print(f"Error: Transaction would revert based on gas estimation. Contract Error: {e}")
        return False
    except Exception as e:
        print(f"Error in gas estimation: {e}")
        return False

    if mode == 'simulate':
        return est_gas_need

    elif mode == 'send_raw':
        # Re-build with final gas limit
        swap_tx_final = swap_router_contract.functions.exactInputSingle(params).build_transaction(tx_build_params)

        signed_tx = w3_instance.eth.account.sign_transaction(swap_tx_final, inp_private_key)

        # The send_signed_transaction function already returns a hex string.
        # Calling w3_instance.to_hex() again on an already hex string is redundant and causes the TypeError.
        tx_hash_hex = send_signed_transaction(w3_instance, signed_tx)

        return tx_hash_hex

    else:
        print("Error: Invalid 'mode'. Use 'send_raw' or 'simulate'.")
        return False

In [16]:
def check_token_balance(w3_instance:Web3,token_list_address:list, inp_address):
  ERC20_ABI = '''
  [
  {"constant":true,"inputs":[{"name":"_owner","type":"address"}],
   "name":"balanceOf","outputs":[{"name":"balance","type":"uint256"}],
   "type":"function"},
  {"constant":true,"inputs":[],"name":"decimals",
   "outputs":[{"name":"","type":"uint8"}],"type":"function"},
  {"constant":true,"inputs":[],"name":"symbol",
   "outputs":[{"name":"","type":"string"}],"type":"function"}
  ]
  '''
  balances = []
  for token in token_list_address:
      contract = w3_instance.eth.contract(address=token, abi=ERC20_ABI)
      bal = contract.functions.balanceOf(inp_address).call()
      dec = contract.functions.decimals().call()
      sym = contract.functions.symbol().call()
      balances.append([sym, bal / (10 ** dec)])
  eth_balance = w3_instance.eth.get_balance(inp_address)
  balances.append(["ETH", w3_instance.from_wei(eth_balance, 'ether')])
  return balances

In [17]:
def calculate_price_by_tick(tick:float,decimal_0:int,decimal_1:int):
  tick=int(tick)
  if tick<0:
    tick=-tick
  return (10**(decimal_0-decimal_1))*(1.0001**tick)

In [18]:
def calculate_price_from_sqrt_price(sqrt_price_raw: float, decimal_0: int, decimal_1: int) -> float:
    normalized_sqrt_price = sqrt_price_raw / (2 ** 96)
    raw_price_math = normalized_sqrt_price ** 2
    decimal_adjustment = decimal_0 - decimal_1
    final_price = raw_price_math * (10 ** decimal_adjustment)
    return final_price

In [19]:
def calculate_tick_by_price(price: float, decimal_0: int, decimal_1: int) -> int:
    if price <= 0:
        raise ValueError("Price must be greater than zero")
    decimal_factor = 10 ** (decimal_0 - decimal_1)
    val_to_log = math.log(price) - math.log(decimal_factor)
    tick_base = math.log(1.0001)
    tick = val_to_log / tick_base
    return int(round(tick))

In [20]:
def calculate_required_amount_and_swap(
    current_price: float,
    price_lower: float,
    price_upper: float,
    amount0_wallet: float,
    amount1_wallet: float
) -> dict:
    if not (0 < price_lower < current_price < price_upper):
        return {
            "status": "Error",
            "message": "prices must be positive and current price should be between lower and upper"
        }

    sqrt_pc = math.sqrt(current_price)
    sqrt_pa = math.sqrt(price_lower)
    sqrt_pb = math.sqrt(price_upper)

    numerator = sqrt_pc - sqrt_pa

    denominator = (1 / sqrt_pc) - (1 / sqrt_pb)
    required_ratio = numerator / denominator
    v_total = (amount0_wallet * current_price) + amount1_wallet

    amount0_target = v_total / (current_price + required_ratio)

    amount1_target = amount0_target * required_ratio


    swap_diff_0 = amount0_wallet - amount0_target
    swap_diff_1 = amount1_wallet - amount1_target
    sell_token='token0'if swap_diff_0>0 else 'token1'
    swap_amount_abs=abs(swap_diff_0) if sell_token=='token0' else abs(swap_diff_1)

    swap_action = ""
    swap_amount_abs = abs(swap_diff_0)

    if swap_diff_0 > 0:
        swap_action = f" {swap_amount_abs:.8f}sell token0 to buy token1 "
    elif swap_diff_0 < 0:
        swap_action = f"{swap_amount_abs:.8f}buy token0 by sell token1 "
    else:
        swap_action = "balance of tokens are ok"

    liquidity_denominator = (1 / sqrt_pc) - (1 / sqrt_pb)
    final_liquidity = amount0_target / liquidity_denominator
    #
    return {
        "status": "Success",
        "final_liquidity_L": final_liquidity,
        "ratio_required": required_ratio,
        "amount0_target": amount0_target,
        "amount1_target": amount1_target,
        "swap_details": swap_action,
        f"swap_amount_{sell_token}": swap_amount_abs
    }


In [21]:
def calculate_new_amount_token_mix(current_price: float, lower: float, upper: float, liquidity: float):
    s = np.sqrt(current_price)
    su = np.sqrt(upper)
    sl = np.sqrt(lower)

    amount_0 = 0.0
    amount_1 = 0.0

    if current_price < lower:
        amount_0 = liquidity * ((1.0 / sl) - (1.0 / su))
        amount_1 = 0.0
    elif current_price > upper:
        amount_0 = 0.0
        amount_1 = liquidity * (su - sl)
    else:
        amount_0 = liquidity * ((1.0 / s) - (1.0 / su))
        amount_1 = liquidity * (s - sl)

    return amount_0, amount_1

In [22]:
def final_true_amount_in_liquidity(current_price: float, lower: float, upper: float, liquidity: float,inp_decimal0,inp_deciaml1):
  amount_0,amount_1=calculate_new_amount_token_mix(current_price,lower,upper,liquidity)
  return amount_0/10**(inp_decimal0-inp_deciaml1),amount_1/10**(inp_deciaml1-inp_decimal0)

In [23]:
V3_LPS[0]

{'token_id': 221855,
 'token0': '0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238',
 'token1': '0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14',
 'fee_tier': 500,
 'liquidity': 535164611370740,
 'tick_range': [187930, 187940],
 'fee0': 0,
 'fee1': 0}

In [24]:
def get_nearest_valid_tick(inp_current_tick, inp_tickspacing):
  return (inp_current_tick // inp_tickspacing) * inp_tickspacing

In [25]:
router_address=w3.to_checksum_address('0x3bFA4769FB09eefC5a80d6E87c3B9C650f7Ae48E')

In [40]:
def check_transaction_status_and_log_gas(w3_instance, tx_hash: str,
                                         inp_time_out:int=60 ):

    try:
        # 1
        print(f"Waiting for {tx_hash} dettail...")
        receipt = w3_instance.eth.wait_for_transaction_receipt(tx_hash, timeout=inp_time_out)

        # 2
        if receipt.status == 1:
            status_text = "SUCCESS"
        else:
            status_text = "FAILED (Reverted)"

        # 3
        gas_used = receipt.gasUsed
        effective_gas_price = receipt.effectiveGasPrice

        #
        transaction_cost_wei = gas_used * effective_gas_price

        print(f"Status: {status_text}")
        print(f"Block Number: {receipt.blockNumber}")
        print(f"Gas Used: {gas_used}")
        print(f"Effective Gas Price (Wei): {effective_gas_price}")
        print(f"Total Cost (Wei): {transaction_cost_wei}")
        print("-" * 30)

        return receipt

    except Exception as e:
        print(f"transaction failed or timed out: {e}")
        return None

In [28]:
def decrease_liquidity_v3_nft(
    inp_position_id,
    inp_liquidity_amount,
    inp_position_manager_contract,
    inp_expected_token0,
    inp_expected_token1,
    w3_instance,
    wallet_address,
    private_key,
    mode: str = 'simulate',
    slippage: float = 0.01):

    # 1
    if slippage > 1 or slippage < 0:
        slippage = 0.01

    # 2
    #
    amount0_min = int(inp_expected_token0 * (1 - slippage))
    amount1_min = int(inp_expected_token1 * (1 - slippage))

    params_decrease = {
        "tokenId": inp_position_id,
        "liquidity": inp_liquidity_amount,
        "amount0Min": amount0_min,
        "amount1Min": amount1_min,
        "deadline": int(time.time()) + 1200
    }

    try:
        # 3
        #
        tx_for_estimate = inp_position_manager_contract.functions.decreaseLiquidity(params_decrease).build_transaction({
            "from": wallet_address,
            "gasPrice": w3_instance.eth.gas_price})

        estimated_gas = w3_instance.eth.estimate_gas(tx_for_estimate)

        #
        safe_gas_limit = int(estimated_gas * 1.1)

        if mode == 'simulate':
            return estimated_gas

        elif mode == 'send_raw':
            # 4
            final_tx = inp_position_manager_contract.functions.decreaseLiquidity(params_decrease).build_transaction({
                "from": wallet_address,
                "nonce": w3_instance.eth.get_transaction_count(wallet_address),
                "gas": safe_gas_limit,
                "gasPrice": w3_instance.eth.gas_price})

            signed_tx = w3_instance.eth.account.sign_transaction(final_tx, private_key)
            tx_hash = w3_instance.eth.send_raw_transaction(signed_tx.raw_transaction)

            return  w3_instance.to_hex(tx_hash)

        else:
            return {"status": False, "error": "Invalid mode"}

    except Exception as e:
        return {"status": False, "error": str(e)}

In [29]:
def collect_liquidity_v3_nft(
    inp_position_id,
    inp_position_manager_contract,
    w3_instance,
    wallet_address,
    private_key,
    mode: str = 'simulate'):

    MAX_UINT128 = 2**128 - 1


    #
    params_collect = {
        "tokenId": inp_position_id,
        "recipient": wallet_address,
        "amount0Max": MAX_UINT128,
        "amount1Max": MAX_UINT128}

    try:
        #
        #
        tx_for_estimate = inp_position_manager_contract.functions.collect(params_collect).build_transaction({
            "from": wallet_address,
            "gasPrice": w3_instance.eth.gas_price})

        estimated_gas = w3_instance.eth.estimate_gas(tx_for_estimate)

        #
        safe_gas_limit = int(estimated_gas * 1.1)

        if mode == 'simulate':
            return estimated_gas

        elif mode == 'send_raw':
            #
            final_tx = inp_position_manager_contract.functions.collect(params_collect).build_transaction({
                "from": wallet_address,
                "nonce": w3_instance.eth.get_transaction_count(wallet_address),
                "gas": safe_gas_limit,
                "gasPrice": w3_instance.eth.gas_price
            })

            #
            signed_tx = w3_instance.eth.account.sign_transaction(final_tx, private_key)
            tx_hash = w3_instance.eth.send_raw_transaction(signed_tx.raw_transaction)

            return w3_instance.to_hex(tx_hash)

        else:
            return {"status": False, "error": "Invalid mode"}

    except Exception as e:
        return {"status": False, "error": str(e)}

In [30]:
def burn_liquidity_v3_nft(
    inp_position_id,
    inp_position_manager_contract,
    w3_instance,
    wallet_address,
    private_key,
    mode: str = 'simulate'
):


    try:
        # 1
        #
        tx_for_estimate = inp_position_manager_contract.functions.burn(inp_position_id).build_transaction({
            "from": wallet_address,
            "gasPrice": w3_instance.eth.gas_price
        })

        estimated_gas = w3_instance.eth.estimate_gas(tx_for_estimate)

        # 2
        safe_gas_limit = int(estimated_gas * 1.1)

        if mode == 'simulate':
            return estimated_gas

        elif mode == 'send_raw':
            # 3
            final_tx = inp_position_manager_contract.functions.burn(inp_position_id).build_transaction({
                "from": wallet_address,
                #
                "nonce": w3_instance.eth.get_transaction_count(wallet_address),
                "gas": safe_gas_limit,
                "gasPrice": w3_instance.eth.gas_price
            })

            #4
            signed_tx = w3_instance.eth.account.sign_transaction(final_tx, private_key)
            tx_hash = w3_instance.eth.send_raw_transaction(signed_tx.raw_transaction)

            return w3_instance.to_hex(tx_hash)

        else:
            return {"status": False, "error": "Invalid mode"}

    except Exception as e:
        return {"status": False, "error": str(e)}

In [31]:
def mint_liquidity_v3_nft(
    token0_contract,
    token1_contract,
    inp_position_manager_contract,
    inp_fee,
    inp_tick_lower,
    inp_tick_upper,
    inp_amount0_desired,
    inp_amount1_desired,
    w3_instance,
    wallet_address,
    private_key,
    mode: str = 'simulate',
    slippage: float = 0.01
):
    # 1
    if slippage > 1 or slippage < 0:
        slippage = 0.01

    # 2
    amount0_min = int(inp_amount0_desired * (1 - slippage))
    amount1_min = int(inp_amount1_desired * (1 - slippage))

    #
    pm_address = inp_position_manager_contract.address

    #
    time.sleep(10)
    current_nonce = w3_instance.eth.get_transaction_count(wallet_address)
    tx_hashes = []

    try:
        # ---------------------------------------------------------
        #
        # ---------------------------------------------------------
        if mode == 'send_raw':
            #
            allowance0 = token0_contract.functions.allowance(wallet_address, pm_address).call()
            if allowance0 < inp_amount0_desired:
                print(f"Approving Token 0...")
                approve_tx0 = token0_contract.functions.approve(pm_address, 2**256 - 1).build_transaction({
                    "from": wallet_address,
                    "nonce": current_nonce,
                    "gasPrice": w3_instance.eth.gas_price
                })
                #
                est_gas0 = w3_instance.eth.estimate_gas(approve_tx0)
                approve_tx0['gas'] = int(est_gas0 * 1.1)

                signed_app0 = w3_instance.eth.account.sign_transaction(approve_tx0, private_key)
                hash0 = w3_instance.eth.send_raw_transaction(signed_app0.raw_transaction)
                tx_hashes.append({"type": "approve_token0", "hash": w3_instance.to_hex(hash0)})

                #
                w3_instance.eth.wait_for_transaction_receipt(hash0)
                current_nonce += 1

            #
            allowance1 = token1_contract.functions.allowance(wallet_address, pm_address).call()
            if allowance1 < inp_amount1_desired:
                print(f"Approving Token 1...")
                approve_tx1 = token1_contract.functions.approve(pm_address, 2**256 - 1).build_transaction({
                    "from": wallet_address,
                    "nonce": current_nonce,
                    "gasPrice": w3_instance.eth.gas_price
                })
                #
                est_gas1 = w3_instance.eth.estimate_gas(approve_tx1)
                approve_tx1['gas'] = int(est_gas1 * 1.1)

                signed_app1 = w3_instance.eth.account.sign_transaction(approve_tx1, private_key)
                hash1 = w3_instance.eth.send_raw_transaction(signed_app1.raw_transaction)
                tx_hashes.append({"type": "approve_token1", "hash": w3_instance.to_hex(hash1)})

                #
                w3_instance.eth.wait_for_transaction_receipt(hash1)
                current_nonce += 1

        # ---------------------------------------------------------
        #
        # ---------------------------------------------------------
        token0_addr = token0_contract.address
        token1_addr = token1_contract.address

        params_mint = {
            "token0": token0_addr,
            "token1": token1_addr,
            "fee": inp_fee,
            "tickLower": inp_tick_lower,
            "tickUpper": inp_tick_upper,
            "amount0Desired": int(inp_amount0_desired),
            "amount1Desired": int(inp_amount1_desired),
            "amount0Min": amount0_min,
            "amount1Min": amount1_min,
            "recipient": wallet_address,
            "deadline": int(time.time()) + 1200
        }

        # ---------------------------------------------------------
        # ---------------------------------------------------------
        # 3
        tx_for_estimate = inp_position_manager_contract.functions.mint(params_mint).build_transaction({
            "from": wallet_address,
            "nonce": current_nonce, #
            "gasPrice": w3_instance.eth.gas_price})

        #
        estimated_gas = w3_instance.eth.estimate_gas(tx_for_estimate)

        #
        safe_gas_limit = int(estimated_gas * 1.1)

        if mode == 'simulate':
            return estimated_gas

        elif mode == 'send_raw':
            # 4
            final_tx = inp_position_manager_contract.functions.mint(params_mint).build_transaction({
                "from": wallet_address,
                "nonce": current_nonce, #
                "gas": safe_gas_limit,
                "gasPrice": w3_instance.eth.gas_price})

            signed_tx = w3_instance.eth.account.sign_transaction(final_tx, private_key)
            mint_hash = w3_instance.eth.send_raw_transaction(signed_tx.raw_transaction)

            tx_hashes.append({"type": "mint", "hash": w3_instance.to_hex(mint_hash)})

            #
            return w3_instance.to_hex(mint_hash)


        else:
            return {"status": False, "error": "Invalid mode"}

    except Exception as e:
        return {"status": False, "error": str(e), "transactions_sent": tx_hashes}

In [32]:
check_liquidity_v3_nft(my_wallet_address,position_manager_contract)

تعداد NFT پوزیشن‌ها: 2


[{'token_id': 221855,
  'token0': '0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238',
  'token1': '0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14',
  'fee_tier': 500,
  'liquidity': 535164611370740,
  'tick_range': [187930, 187940],
  'fee0': 0,
  'fee1': 0},
 {'token_id': 221856,
  'token0': '0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238',
  'token1': '0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14',
  'fee_tier': 100,
  'liquidity': 19436422026195112,
  'tick_range': [187953, 187955],
  'fee0': 0,
  'fee1': 0}]

In [33]:
check_token_balance(w3,['0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238','0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14'],my_wallet_address)

[['USDC', 285.900541],
 ['WETH', 0.029582085970823645],
 ['ETH', Decimal('0.285628968374646189')]]

In [34]:
def V3_JIT_REBALANCING(w3_instance,inp_v3_lp,wallet_address,
                       inp_router_address,inp_position_manager_contract,private_key):
  transaction_log=[]
  pool_address=factory.functions.getPool(inp_v3_lp[0]['token0'],inp_v3_lp['token1'],inp_v3_lp['fee_tier']).call()
  pool_abi=fetch_abi_retry(pool_addr,etherscan_apikey,CHAIN_ID)
  poolv_contract=w3.eth.contract(address=pool_addr, abi=pool_abi)
  pool_info=pool_current_info(poolv_contract)
  token0_address=w3.to_checksum_address(pool_info['token0_address'])
  token1_address=w3.to_checksum_address(pool_info['token1_address'])
  erc20_abi=fetch_abi_retry("0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14",etherscan_apikey,CHAIN_ID)
  token0_contract=w3.eth.contract(address=token0_address, abi=erc20_abi)
  token1_contract=w3.eth.contract(address=token1_address, abi=erc20_abi)
  decimal0=token0_contract.functions.decimals().call()
  decimal1=token1_contract.functions.decimals().call()

  cp=calculate_price_by_tick(pool_info['current_tick'],decimal0,decimal1)
  lp=calculate_price_by_tick(inp_v3_lp['tick_range'][0],decimal0,decimal1)
  up=calculate_price_by_tick(inp_v3_lp['tick_range'][1],decimal0,decimal1)
  am0,am1=final_true_amount_in_liquidity(cp,lp,up,V3_LPS[0]['liquidity'],decimal0,decimal1)

  if pool_info['current_tick']<inp_v3_lp['tick_range'][0] or pool_info['current_tick']>inp_v3_lp['tick_range'][1]:

    d_hash=decrease_liquidity_v3_nft(inp_v3_lp['token_id'],inp_v3_lp['liquidity'],
        position_manager_contract,am0*10**decimal0,am1*10**decimal1,w3,my_wallet_address,PRIVATE_KEY,'send_raw')
    d_log=check_transaction_status_and_log_gas(w3_instance,d_hash,30)
    transaction_log.append({'decrease':{'hash':d_hash,'log':d_log}})

    time.sleep(1)
    c_hash=collect_liquidity_v3_nft(inp_v3_lp['token_id'],position_manager_contract,w3,my_wallet_address,PRIVATE_KEY,'send_raw')
    c_log=check_transaction_status_and_log_gas(w3_instance,c_hash,30)
    transaction_log.append({'collect':{'hash':c_hash,'log':c_log}})

    time.sleep(1)
    b_hash=burn_liquidity_v3_nft(inp_v3_lp['token_id'],position_manager_contract,w3,my_wallet_address,PRIVATE_KEY,'send_raw')
    b_log=check_transaction_status_and_log_gas(w3_instance,b_hash,30)
    transaction_log.append({'burn':{'hash':b_hash,'log':b_log}})
    time.sleep(1)

    pool_info=pool_current_info(poolv_contract)
    ctb=check_token_balance([pool_info['token0_address'],pool_info['token1_address']],wallet_address)

    lowertick,uppertick=fix_get_true_tick_range(pool_info['current_tick'],pool_info['tick_spacing'],inp_divid=4)
    price_lower=calculate_price_by_tick(lowertick,decimal0,decimal1)
    price_upper=calculate_price_by_tick(uppertick,decimal0,decimal1)
    cp=calculate_price_by_tick(pool_info['current_tick'],decimal0,decimal1)
    res=calculate_required_amount_and_swap(cp,price_lower,price_upper,ctb[0][1],ctb[1][1])

    hx=''
    if res['amount0_target']-ctb[0][1]<0 and res['amount1_target']-ctb[1][1]>0:

        hx=make_swap(token0_address,token1_address,np.abs(res['amount0_target']-ctb[0][1]),
              wallet_address,pool_addr,inp_router_address,'send_raw',private_key)

    elif res['amount0_target']-ctb[0][1]>0 and res['amount1_target']-ctb[1][1]<0:

          hx=make_swap(token1_address,token0_address,np.abs(res['amount1_target']-ctb[1][1]),
              wallet_address,pool_addr,inp_router_address,'send_raw',private_key)

    time.sleep(10)
    swap_log=check_transaction_status_and_log_gas(w3_instance,hx,30)
    transaction_log.append({'swap':{'hash':hx,'log':swap_log}})

    ctb=check_token_balance([pool_info['token0_address'],pool_info['token1_address']],my_wallet_address)
    final_amount_0=ctb[0][1]
    final_amount_1=ctb[1][1]

    m_hash=mint_liquidity_v3_nft(token0_contract,token1_contract,inp_position_manager_contract,
                          pool_info['fee_tier'],lowertick,uppertick,final_amount_0,
                          final_amount_1,w3_instance,wallet_address,private_key,'send_raw')
    m_log=check_transaction_status_and_log_gas(w3_instance,m_hash,30)
    transaction_log.append({'mint':{'hash':m_hash,'log':m_log}})
    if m_log['status']==1:
      print('Done')
      return transaction_log
    else:
      return False

In [35]:
def fix_get_true_tick_range(inp_tick:int,inp_tickspacing:int,inp_divid:int):
  a=get_nearest_valid_tick(inp_tick,inp_tickspacing)
  b=a+(1*inp_tickspacing)
  #print(a,b)
  lowertick=0
  uppertick=0
  st=(b-a)/inp_divid
  if inp_tick>(a+st*(inp_divid-1)):
    lowertick=a
    uppertick=b+(1*inp_tickspacing)
  elif inp_tick<(a+1*st):
    lowertick=a-(1*inp_tickspacing)
    uppertick=b
  else:
    lowertick=a
    uppertick=b
  return lowertick,uppertick

In [36]:
get_nearest_valid_tick(187935,10)

187930

In [37]:
fix_get_true_tick_range(187935,10,4)

(187930, 187940)

In [42]:
check_transaction_status_and_log_gas(w3,'c1402c026a12026dcde6b2dd979be6f833099b491bd06c3d6fd021903026f0f0')

Waiting for c1402c026a12026dcde6b2dd979be6f833099b491bd06c3d6fd021903026f0f0 dettail...
Status: SUCCESS
Block Number: 9834184
Gas Used: 118707
Effective Gas Price (Wei): 1000010
Total Cost (Wei): 118708187070
------------------------------


AttributeDict({'type': 2,
 'status': 1,
 'cumulativeGasUsed': 20562747,
 'logs': [AttributeDict({'address': '0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238',
   'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'),
    HexBytes('0x0000000000000000000000003289680dd4d6c10bb19b899729cda5eef58aeff1'),
    HexBytes('0x0000000000000000000000007868edefc36a74eba220c75acfb05c59a4e55822')],
   'data': HexBytes('0x000000000000000000000000000000000000000000000000000000000415475b'),
   'blockHash': HexBytes('0x5877fcb841c0b8c7c2d7f1d3a4dd0d9f125a6b815914408c4bb17a463ae7f179'),
   'blockNumber': 9834184,
   'blockTimestamp': '0x693dda6c',
   'transactionHash': HexBytes('0xc1402c026a12026dcde6b2dd979be6f833099b491bd06c3d6fd021903026f0f0'),
   'transactionIndex': 103,
   'logIndex': 423,
   'removed': False}),
  AttributeDict({'address': '0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14',
   'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b

In [43]:
transaction_log=[]
for lp_nft in V3_LPS:

  pool_addr=factory.functions.getPool(
      lp_nft['token0'],lp_nft['token1'],lp_nft['fee_tier']).call()
  pool_abi=fetch_abi_retry(pool_addr,etherscan_apikey,CHAIN_ID)
  poolv_contract=w3.eth.contract(address=pool_addr, abi=pool_abi)
  pool_info=pool_current_info(poolv_contract)
  token0_address=w3.to_checksum_address(pool_info['token0_address'])
  token1_address=w3.to_checksum_address(pool_info['token1_address'])
  erc20_abi=fetch_abi_retry("0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14",etherscan_apikey,CHAIN_ID)
  token0_contract=w3.eth.contract(address=token0_address, abi=erc20_abi)
  token1_contract=w3.eth.contract(address=token1_address, abi=erc20_abi)
  decimal0=token0_contract.functions.decimals().call()
  decimal1=token1_contract.functions.decimals().call()
  symbol0 = token0_contract.functions.symbol().call()
  symbol1 = token1_contract.functions.symbol().call()
  print(symbol0, decimal0, symbol1, decimal1)
  print('pool_info :',pool_info)

  cp=calculate_price_by_tick(pool_info['current_tick'],decimal0,decimal1)
  lower_price=calculate_price_by_tick(lp_nft['tick_range'][0],decimal0,decimal1)
  up=calculate_price_by_tick(lp_nft['tick_range'][1],decimal0,decimal1)
  #print(cp,lower_price,up)
  am0,am1=calculate_new_amount_token_mix(cp,lower_price,up,lp_nft['liquidity'])
  #print(am0,am1)
  if pool_info['current_tick']<lp_nft['tick_range'][0] or pool_info['current_tick']>lp_nft['tick_range'][1]:

    d_hash=decrease_liquidity_v3_nft(lp_nft['token_id'],lp_nft['liquidity'],
        position_manager_contract,int(am0),int(am1),w3,my_wallet_address,PRIVATE_KEY,'send_raw')
    print(d_hash)
    time.sleep(10)
    d_log=check_transaction_status_and_log_gas(w3,d_hash)
    transaction_log.append({'decrease':{'hash':d_hash,'log':d_log}})
    time.sleep(10)

    c_hash=collect_liquidity_v3_nft(lp_nft['token_id'],position_manager_contract,w3,my_wallet_address,PRIVATE_KEY,'send_raw')
    print(c_hash)
    time.sleep(10)
    c_log=check_transaction_status_and_log_gas(w3,c_hash)
    transaction_log.append({'collect':{'hash':c_hash,'log':c_log}})
    time.sleep(10)

    b_hash=burn_liquidity_v3_nft(lp_nft['token_id'],position_manager_contract,w3,my_wallet_address,PRIVATE_KEY,'send_raw')
    print(b_hash)
    time.sleep(10)
    b_log=check_transaction_status_and_log_gas(w3,b_hash)
    transaction_log.append({'burn':{'hash':b_hash,'log':b_log}})
    time.sleep(10)


    pool_info=pool_current_info(poolv_contract)
    pool_tokens=[pool_info['token0_address'],pool_info['token1_address']]
    ctb=check_token_balance(w3,pool_tokens,my_wallet_address)
    lowertick,uppertick=fix_get_true_tick_range(pool_info['current_tick'],pool_info['tick_spacing'],inp_divid=4)
    #print(lowertick,uppertick)
    #print(pool_info['current_tick'])
    #print(pool_info['tick_spacing'])
    price_lower=calculate_price_by_tick(lowertick,decimal0,decimal1)
    price_upper=calculate_price_by_tick(uppertick,decimal0,decimal1)
    cp=calculate_price_by_tick(pool_info['current_tick'],decimal0,decimal1)
    #print(cp,price_lower,price_upper)
    res=calculate_required_amount_and_swap(cp,price_lower,price_upper,ctb[0][1],ctb[1][1])
    print(res)
    hx=''
    if res['amount0_target']-ctb[0][1]<0 and res['amount1_target']-ctb[1][1]>0:

        hx=make_swap(w3, token0_address,token1_address,np.abs(res['amount0_target']-ctb[0][1]),
              my_wallet_address,pool_addr,router_address,'send_raw',PRIVATE_KEY)

    elif res['amount0_target']-ctb[0][1]>0 and res['amount1_target']-ctb[1][1]<0:

          hx=make_swap(w3, token1_address,token0_address,np.abs(res['amount1_target']-ctb[1][1]),
              my_wallet_address,pool_addr,router_address,'send_raw',PRIVATE_KEY)

    time.sleep(10)
    print(hx)
    s_log=check_transaction_status_and_log_gas(w3,hx)
    transaction_log.append({'swap':{'hash':hx,'log':s_log}})
    time.sleep(10)

    ctb=check_token_balance(w3,pool_tokens,my_wallet_address)
    final_amount_0=ctb[0][1]
    final_amount_1=ctb[1][1]
    final_amount_0=int(final_amount_0 * (10**decimal0))
    final_amount_1=int(final_amount_1 * (10**decimal1))

    m_hash=mint_liquidity_v3_nft(token0_contract,token1_contract,
                                 position_manager_contract,pool_info['fee_tier'],
                                 lowertick,uppertick,final_amount_0,final_amount_1,w3,my_wallet_address,PRIVATE_KEY,'send_raw')
    time.sleep(10)
    m_log=check_transaction_status_and_log_gas(w3,m_hash)
    print(m_hash)
    transaction_log.append({'mint':{'hash':m_hash,'log':m_log}})

Successfully fetched ABI from: 0x3289680dD4d6C10bb19b899729cda5eEF58AEfF1
Successfully fetched ABI from: 0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14
USDC 6 WETH 18
pool_info : {'token0_address': '0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238', 'token1_address': '0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14', 'sqrtprice': 964783687166113610903994275106874, 'current_tick': 188155, 'liquidity': 11507674624337212, 'fee_tier': 500, 'tick_spacing': 10}
0x12dc3ba2ef34a78336c0aff0ada5836b6f959ac773004872eaf10c3d57e33081
Waiting for 0x12dc3ba2ef34a78336c0aff0ada5836b6f959ac773004872eaf10c3d57e33081 dettail...
Status: SUCCESS
Block Number: 9841139
Gas Used: 154333
Effective Gas Price (Wei): 1005927
Total Cost (Wei): 155247731691
------------------------------
0xefffce86ac6f3fb2cc8797e7e4c023608ce2bd608b01edcc9f404021bcd9397c
Waiting for 0xefffce86ac6f3fb2cc8797e7e4c023608ce2bd608b01edcc9f404021bcd9397c dettail...
Status: SUCCESS
Block Number: 9841141
Gas Used: 95027
Effective Gas Price (Wei): 1005928


In [44]:
transaction_log


[{'decrease': {'hash': '0x12dc3ba2ef34a78336c0aff0ada5836b6f959ac773004872eaf10c3d57e33081',
   'log': AttributeDict({'type': 0,
    'status': 1,
    'cumulativeGasUsed': 29073675,
    'logs': [AttributeDict({'address': '0x3289680dD4d6C10bb19b899729cda5eEF58AEfF1',
      'topics': [HexBytes('0x0c396cd989a39f4459b5fa1aed6a9a8dcdbc45908acfd67e028cd568da98982c'),
       HexBytes('0x0000000000000000000000001238536071e1c677a632429e3655c799b22cda52'),
       HexBytes('0x000000000000000000000000000000000000000000000000000000000002de1a'),
       HexBytes('0x000000000000000000000000000000000000000000000000000000000002de24')],
      'data': HexBytes('0x0000000000000000000000000000000000000000000000000001e6bab88e9ef40000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000b72d0cb175422'),
      'blockHash': HexBytes('0xbf273a77e7cfdd23e8968be0fcd20f526b90d1a2aed78ca366307983d335eee5'),
      'blockNumber': 9841139,
      'blockTimestamp': 

In [ ]:
check_liquidity_v3_nft(my_wallet_address,position_manager_contract)

تعداد NFT پوزیشن‌ها: 0
lp nft not found
